    Algoritmo de regras

    Base Risco de Crédito

In [2]:
import Orange

In [6]:
base_risco_credito = Orange.data.Table('./assets/risco_credito_regras.csv')
base_risco_credito

[[ruim, alta, nenhuma, 0_15 | alto],
 [desconhecida, alta, nenhuma, 15_35 | alto],
 [desconhecida, baixa, nenhuma, 15_35 | moderado],
 [desconhecida, baixa, nenhuma, acima_35 | alto],
 [desconhecida, baixa, nenhuma, acima_35 | baixo],
 ...
]

In [7]:
base_risco_credito.domain

[historia, divida, garantias, renda | risco]

In [9]:
cn2 = Orange.classification.rules.CN2Learner()
regras_risco_credito = cn2(base_risco_credito)

In [10]:
for regras in regras_risco_credito.rule_list:
    print(regras)

IF renda==0_15 THEN risco=alto 
IF historia==boa AND divida!=alta THEN risco=baixo 
IF historia==boa AND garantias!=nenhuma THEN risco=baixo 
IF historia==boa AND renda!=15_35 THEN risco=baixo 
IF historia==boa THEN risco=moderado 
IF divida==alta THEN risco=alto 
IF historia!=desconhecida THEN risco=moderado 
IF garantias==adequada THEN risco=baixo 
IF renda==15_35 THEN risco=moderado 
IF historia==desconhecida THEN risco=baixo 
IF TRUE THEN risco=alto 


In [12]:
#historia boa (0), divida alta (0), garantias nenhuma (1), renda > 35 (2)]
#historia ruim (2), divida alta (0), garantia adequada (0), renda < 15 (0)
previsoes = regras_risco_credito([['boa', 'alta', 'nenhuma', 'acima_35'], ['ruim', 'alta', 'adequada', '0_15']])
previsoes

array([1, 0])

In [15]:
base_risco_credito.domain.class_var.values

('alto', 'baixo', 'moderado')

In [17]:
for i in previsoes:
    #print(i)
    print(base_risco_credito.domain.class_var.values[i])

baixo
alto


    Base credit data

In [3]:
base_credit = Orange.data.Table('./assets/credit_data_regras.csv')

In [19]:
base_credit.domain

[income, age, loan | default]

In [20]:
base_dividida = Orange.evaluation.testing.sample(base_credit, n=0.25)

In [21]:
base_dividida

([[67802.7, 49.038, 13443.5 | 0],
  [46166.2, 25.9961, 901.753 | 0],
  [54077.8, 57.1207, 7149.07 | 0],
  [20146, 21.2828, 839.839 | 0],
  [51733.3, 24.915, 7906.14 | 1],
  ...
 ],
 [[32804.9, 62.2944, 4961.26 | 0],
  [44488.2, 33.4312, 2751.09 | 0],
  [60103, 56.0538, 2632.27 | 0],
  [66370.7, 38.4055, 5906.03 | 0],
  [45540.3, 59.3181, 1490.47 | 0],
  ...
 ])

In [22]:
base_treinamento = base_dividida[1]
base_teste = base_dividida[0]

In [23]:
regras_credit = cn2(base_treinamento)

In [24]:
for regras in regras_credit.rule_list:
    print(regras)

IF age>=34.9966813726351 THEN default=0 
IF loan<=2495.13299137587 AND income>=20568.891305010802 THEN default=0 
IF loan>=8045.4409532236705 THEN default=1 
IF income>=58132.4712652713 AND age>=22.939635145478 THEN default=0 
IF income>=58609.13148382679 AND age>=22.918212262913602 THEN default=1 
IF income>=58609.13148382679 THEN default=0 
IF loan>=5862.83302915672 AND age>=26.854012909811 THEN default=1 
IF loan>=6361.97343810153 THEN default=1 
IF income<=31722.7309499867 AND loan>=3343.81635769923 THEN default=1 
IF income>=50360.6787929311 THEN default=0 
IF income<=22089.8374845274 AND age>=21.3656869572587 THEN default=1 
IF loan<=5416.3577980789005 AND income>=44299.3717431091 THEN default=0 
IF loan<=4849.33378471958 AND income>=38129.754869733 THEN default=0 
IF age>=34.7410444347188 THEN default=0 
IF age>=34.669146894011604 THEN default=1 
IF age>=34.1016539284028 THEN default=0 
IF age>=33.6895613595843 THEN default=1 
IF loan>=4285.38691174949 AND income>=50289.66474812

In [25]:
previsoes_credit = Orange.evaluation.testing.TestOnTestData(base_treinamento, base_teste, [lambda testdata: regras_credit])

In [27]:
previsoes_credit

In [28]:
Orange.evaluation.CA(previsoes_credit)

array([0.974])

    Base Census

In [4]:
base_census = Orange.data.Table('./assets/census_regras.csv')

In [31]:
base_census.domain

[age, workclass, final-weight, education, education-num, marital-status, occupation, relationship, race, sex, capital-gain, capital-loos, hour-per-week, native-country | income]

In [32]:
base_census_dividida = Orange.evaluation.testing.sample(base_census, n=0.25)

In [33]:
base_census_treinamento = base_census_dividida[1]
base_census_teste = base_census_dividida[0]

In [34]:
regras_census = cn2(base_census_treinamento)

In [37]:
previsoes_census = Orange.evaluation.testing.TestOnTestData(base_census_treinamento, base_census_teste, [lambda testdata: regras_census])

In [38]:
Orange.evaluation.CA(previsoes_census)

array([0.82469287])

    Majority Learner

In [5]:
base_credit.domain

[income, age, loan | default]

In [6]:
majority_credit = Orange.classification.MajorityLearner()

In [7]:
previsoes_ml_credit = Orange.evaluation.testing.TestOnTestData(base_credit, base_credit, [majority_credit])

In [8]:
Orange.evaluation.CA(previsoes_ml_credit)

array([0.8585])

In [10]:
#Census
majority_census = Orange.classification.MajorityLearner()

In [11]:
previsoes_ml_census = Orange.evaluation.testing.TestOnTestData(base_census, base_census, [majority_census])

In [12]:
Orange.evaluation.CA(previsoes_ml_census)

array([0.75919044])